In [2]:
import os
import glob
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
import matplotlib.pyplot as plt
import csv

from neuralop.models import FNO2d, TFNO2d, UNO, LocalFNO, FNO3d, TFNO3d
from spatioTemporalFNO import SpatioTemporalFNO

In [5]:
image_path = "/cs/student/projects1/2021/rstewart/denseflow/build/love_parade/flow_x_00000.jpg"
img = Image.open(image_path)
width, height = img.size
print("Width:", width, "Height:", height)


Width: 2460 Height: 1338


In [10]:
neural_operator = "fno"
sequence_length = 1

In [ ]:
# Define the CSV file and write a header.
if sequence_length == 1:
    csv_file = "training_history_"+neural_operator+"_bull2.csv"
else:
    csv_file = "training_history_"+neural_operator+"_bull2_"+str(sequence_length)+".csv"
with open(csv_file, mode="w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Epoch", "MSE Loss"])

In [7]:
torch.cuda.empty_cache()

In [8]:
import os
import glob
import numpy as np
import torch
from torch.utils.data import Dataset
from PIL import Image

class OpticalFlowDataset(Dataset):
    """
    Loads optical flow images for forecasting.
    
    This dataset can operate in two modes:
      - Original Mode (single-frame input): when sequence_length == 1.
        Returns:
           input  = optical flow at time t (shape: (C, H, W))
           target = optical flow at time t + forecast_offset (shape: (C, H, W))
      - Sequence Mode (multi-frame input): when sequence_length > 1.
        Returns:
           input  = sequence of optical flow frames of length `sequence_length` 
                    (shape: (T, C, H, W))
           target = optical flow at time t + sequence_length + forecast_offset - 1
                    (shape: (C, H, W))
                    
    The dataset length is computed appropriately based on the mode.
    """
    def __init__(self, data_dir, transform=None, sequence_length=1, forecast_offset=1):
        self.data_dir = data_dir
        self.transform = transform
        self.sequence_length = sequence_length  # Set to 1 for original behavior; >1 for multi-frame input.
        self.forecast_offset = forecast_offset
        
        self.samples = []
        flow_x_files = sorted(glob.glob(os.path.join(data_dir, 'flow_x_*.jpg')))
        print("Found flow_x files:", flow_x_files)
        for fx in flow_x_files:
            fy = fx.replace("flow_x_", "flow_y_")
            if os.path.exists(fy):
                self.samples.append((fx, fy))
            else:
                print(f"Warning: Matching flow_y image not found for {fx}")

    def __len__(self):
        if self.sequence_length == 1:
            # Original: need one frame for input plus forecast_offset for target.
            return len(self.samples) - self.forecast_offset
        else:
            # Sequence mode: need sequence_length frames for input plus forecast_offset.
            return len(self.samples) - (self.sequence_length - 1 + self.forecast_offset)

    def __getitem__(self, idx):
        if self.sequence_length == 1:
            # ----- Original Mode -----
            # Input frame at time t
            flow_x_file, flow_y_file = self.samples[idx]
            input_flow_x = np.array(Image.open(flow_x_file).convert('L'), dtype=np.float32) / 255.0
            input_flow_y = np.array(Image.open(flow_y_file).convert('L'), dtype=np.float32) / 255.0
            input_flow = np.stack([input_flow_x, input_flow_y], axis=-1)  # (H, W, 2)
            
            # Target frame at time t + forecast_offset
            target_flow_x_file, target_flow_y_file = self.samples[idx + self.forecast_offset]
            target_flow_x = np.array(Image.open(target_flow_x_file).convert('L'), dtype=np.float32) / 255.0
            target_flow_y = np.array(Image.open(target_flow_y_file).convert('L'), dtype=np.float32) / 255.0
            target_flow = np.stack([target_flow_x, target_flow_y], axis=-1)
            
            if self.transform is not None:
                input_flow = self.transform(input_flow)
                target_flow = self.transform(target_flow)
            
            # Convert to tensor and change order to (C, H, W)
            input_tensor = torch.tensor(input_flow, dtype=torch.float32).permute(2, 0, 1)
            target_tensor = torch.tensor(target_flow, dtype=torch.float32).permute(2, 0, 1)
            
            return input_tensor, target_tensor
        
        else:
            # ----- Sequence Mode -----
            # Build the input sequence from idx to idx + sequence_length - 1
            input_frames = []
            for i in range(self.sequence_length):
                flow_x_file, flow_y_file = self.samples[idx + i]
                input_flow_x = np.array(Image.open(flow_x_file).convert('L'), dtype=np.float32) / 255.0
                input_flow_y = np.array(Image.open(flow_y_file).convert('L'), dtype=np.float32) / 255.0
                flow = np.stack([input_flow_x, input_flow_y], axis=-1)  # (H, W, 2)
                if self.transform is not None:
                    flow = self.transform(flow)
                flow_tensor = torch.tensor(flow, dtype=torch.float32).permute(2, 0, 1)
                input_frames.append(flow_tensor)
            # Stack the sequence into a tensor: (T, C, H, W)
            input_tensor = torch.stack(input_frames, dim=0)
            
            # Target frame: located at index = idx + sequence_length + forecast_offset - 1
            target_idx = idx + self.sequence_length + self.forecast_offset - 1
            target_flow_x_file, target_flow_y_file = self.samples[target_idx]
            target_flow_x = np.array(Image.open(target_flow_x_file).convert('L'), dtype=np.float32) / 255.0
            target_flow_y = np.array(Image.open(target_flow_y_file).convert('L'), dtype=np.float32) / 255.0
            target_flow = np.stack([target_flow_x, target_flow_y], axis=-1)
            if self.transform is not None:
                target_flow = self.transform(target_flow)
            target_tensor = torch.tensor(target_flow, dtype=torch.float32).permute(2, 0, 1)
            
            return input_tensor, target_tensor


In [13]:
seed = 42
torch.manual_seed(seed)

data_dir = '/cs/student/projects1/2021/rstewart/denseflow/build/love_parade/' 

# Create dataset with forecast_offset=1 (predicting next frame)
dataset = OpticalFlowDataset(data_dir, forecast_offset=1, sequence_length=sequence_length)
print("Total available samples for forecasting:", len(dataset)) 

# Randomly split the dataset into training and testing
train_ratio = 0.8
train_size = int(train_ratio * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size], generator=torch.Generator().manual_seed(seed))

print("Train dataset size:", len(train_dataset))
print("Test dataset size:", len(test_dataset))

# Create DataLoaders
batch_size = 1  # Adjust as needed
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)



Found flow_x files: ['/cs/student/projects1/2021/rstewart/denseflow/build/love_parade/flow_x_00000.jpg', '/cs/student/projects1/2021/rstewart/denseflow/build/love_parade/flow_x_00001.jpg', '/cs/student/projects1/2021/rstewart/denseflow/build/love_parade/flow_x_00003.jpg', '/cs/student/projects1/2021/rstewart/denseflow/build/love_parade/flow_x_00004.jpg', '/cs/student/projects1/2021/rstewart/denseflow/build/love_parade/flow_x_00005.jpg', '/cs/student/projects1/2021/rstewart/denseflow/build/love_parade/flow_x_00006.jpg', '/cs/student/projects1/2021/rstewart/denseflow/build/love_parade/flow_x_00008.jpg', '/cs/student/projects1/2021/rstewart/denseflow/build/love_parade/flow_x_00010.jpg', '/cs/student/projects1/2021/rstewart/denseflow/build/love_parade/flow_x_00011.jpg', '/cs/student/projects1/2021/rstewart/denseflow/build/love_parade/flow_x_00013.jpg', '/cs/student/projects1/2021/rstewart/denseflow/build/love_parade/flow_x_00015.jpg', '/cs/student/projects1/2021/rstewart/denseflow/build/lo

In [ ]:

if neural_operator == "fno" and sequence_length == 1:
    modes1 = 12  
    modes2 = 12  
    hidden_channels = 32

    model = FNO2d(
        n_modes_height=modes1,
        n_modes_width=modes2,
        hidden_channels=hidden_channels,
        in_channels=2,   
        out_channels=2  
    )

if neural_operator == "fno" and sequence_length != 1:
    modes1 = 12
    modes2 = 12  
    hidden_channels = 32
    
    model = FNO3d(
        n_modes_height=modes1,
        n_modes_width=modes2,
        n_modes_depth=4,
        hidden_channels=hidden_channels,
        in_channels=2,   
        out_channels=2  
    )

if neural_operator == "tfno":
    modes1 = 12  
    modes2 = 12  
    hidden_channels = 32

    model = TFNO2d(
        n_modes_height=modes1,
        n_modes_width=modes2,
        hidden_channels=hidden_channels,
        in_channels=2,   
        out_channels=2  
    )

if neural_operator == "uno":
    in_channels = 2
    out_channels = 2

    model = UNO(
        in_channels=in_channels,
        out_channels=out_channels,
        hidden_channels=32,  
        n_layers=2,         
        uno_out_channels=[32, 64],
        uno_n_modes=[(12, 12), (12, 12)],
        uno_scalings=[[1, 1], [1, 1]]
    )

if neural_operator == "localfno":
    in_channels = 2   
    out_channels = 2  
    kernel_size = 3   
    n_layers = 4      
    hidden_channels = 32
    n_dim = 2         
    modes1 = 12  
    modes2 = 12  
    
    model = LocalFNO(
        n_modes=(modes1, modes2),
        default_in_shape=(height, width),
        in_channels=in_channels,
        out_channels=out_channels,
        kernel_size=kernel_size,
        n_layers=n_layers,
        hidden_channels=hidden_channels,
        n_dim=n_dim,
    )

if neural_operator == "spatiotemporalfno":
    modes1 = 12  
    modes2 = 12 
    hidden_channels = 32
    T_in = 3   
    
    model = SpatioTemporalFNO(
        n_modes=(modes1, modes2),
        in_channels=2,       
        out_channels=2,       
        hidden_channels=hidden_channels,
        n_layers=4,           
        T_in=T_in,            
        temporal_kernel=3    
    )

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print("Operator: ",neural_operator)
print(device)

Operator:  fno
cuda


In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        inputs = inputs.to(device)  
        targets = targets.to(device)  
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}] MSE Loss: {avg_loss:.6f}")

    with open(csv_file, mode="a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([epoch+1, avg_loss])

if sequence_length == 1:
    model_save_path = neural_operator+"_model.pth"
else:
    model_save_path = neural_operator+"_model_"+str(sequence_length)+".pth"

torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")



In [ ]:
###########################################################################
# 5. Training Loop
###########################################################################

num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        # inputs has shape (B, T=3, C=2, H, W).
        # We want (B, C=2, depth=3, H, W) for the FNO3d.
        inputs = inputs.permute(0, 2, 1, 3, 4)  # => (B, 2, 3, H, W)
          # add a singleton depth dimension so it becomes (B, 2, 1, H, W)
        targets = targets.unsqueeze(2).to(device)               # (B, 2, 1, H, W)

        inputs = inputs.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)  # now the second dimension is in_channels=2
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}] MSE Loss: {avg_loss:.6f}")

    # Append the current epoch and average loss to the CSV file.
    with open(csv_file, mode="a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([epoch+1, avg_loss])

# Define model save path
if sequence_length == 1:
    model_save_path = neural_operator+"_model.pth"
else:
    model_save_path = neural_operator+"_model_"+str(sequence_length)+".pth"

# After training is complete
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")



In [ ]:
model_load_path = "/cs/student/projects1/2021/rstewart/code/models/love_parade_3_length/spatiotemporalfno_model_3.pth"
model.load_state_dict(torch.load(model_load_path, map_location="cpu"))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

model.eval()
total_loss = 0
with torch.no_grad():
    for idx, (inputs, targets) in enumerate(test_loader):
        inputs = inputs.to(device)
        targets = targets.to(device)

        outputs = model(inputs)
        loss_val = criterion(outputs, targets).item()
        total_loss += loss_val

        print(f"Test sample {idx} MSE: {loss_val:.6f}")

avg_loss = total_loss / len(test_loader)
print(f"Average test MSE: {avg_loss:.6f}")
        


In [ ]:
mse_fno = []

In [ ]:
mse_stfno = []

In [ ]:
import torch
import numpy as np
from torch.nn import MSELoss

criterion = MSELoss(reduction="none")  # so we can compute per-sample losses
model_load_path = "/cs/student/projects1/2021/rstewart/code/models/bull1_1_length/fno_model.pth"
model.load_state_dict(torch.load(model_load_path, map_location="cpu"))

# 3) Move to device, set to eval
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


model.eval()
with torch.no_grad():
    for inputs, targets in test_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)

        out_fno = model(inputs)
        

        # per-sample MSE (averaged over channels, H, W)
        # criterion gives [B, C, H, W]; we mean over (1,2,3)
        mse_fno.append(criterion(out_fno, targets).mean(dim=(1,2,3)).cpu().numpy())




In [ ]:
mse_fno = np.concatenate(mse_fno)   # shape (N,)

In [ ]:
mse_stfno = np.concatenate(mse_stfno)   # shape (N,)

In [ ]:
from scipy.stats import ttest_rel

t_stat, p_one = ttest_rel(mse_fno, mse_stfno, alternative="greater")
print(f"t = {t_stat:.3f}, one‑sided p = {p_one:.3e}")



In [ ]:
import os
import torch
import matplotlib.pyplot as plt
from PIL import Image 
import sys 

def tensor_to_numpy(tensor, is_input_sequence=False):
    if tensor.is_sparse:
        tensor = tensor.to_dense()

    # Assuming batch_size is 1 for visualization loop
    # Remove batch dim: shape becomes (S, C, H, W) OR (C, S_out, H, W) OR (C, H, W)
    tensor = tensor.squeeze(0)

    if is_input_sequence:
        # Input sequence: Expected (S, C, H, W) -> permute to (S, H, W, C)
        if tensor.ndim != 4:
             print(f"Warning: Expected input sequence tensor shape (S, C, H, W) after batch squeeze, but got {tensor.shape}")
             # Attempt to proceed, but permutation might fail
        return tensor.permute(0, 2, 3, 1).cpu().numpy()
    else:
        # Target/Output: Expected (C, H, W) or (C, 1, H, W) or (C, S_out, H, W) but we only want (C, H, W)
        # If it has 4 dimensions, we assume it's (C, S_out, H, W) and we should only process the first frame (S_out=0)
        if tensor.ndim == 4:
             if tensor.shape[1] == 1:
                 tensor = tensor.squeeze(1) 
             else:
                 print(f"Error: Non-input tensor has unexpected shape (C, S_out, H, W) with S_out > 1 after batch squeeze: {tensor.shape}. This function expects (C, H, W) or (C, 1, H, W) for non-inputs.")

        if tensor.ndim == 3:
             return tensor.permute(1, 2, 0).cpu().numpy()
        else:
             print(f"Error: Final non-input tensor shape not (C, H, W) for permute: {tensor.shape}")
             # Fallback: Return the tensor as is after CPU transfer, without the expected (H, W, C) shape
             return tensor.cpu().numpy()


# Define the model loading path
model_load_path = "/cs/student/projects1/2021/rstewart/code/models/love_parade_3_length/fno_model_3.pth"
model_folder_name = os.path.basename(os.path.dirname(model_load_path))
filename_stem = os.path.splitext(os.path.basename(model_load_path))[0]
model_type_name = filename_stem.split('_model')[0] if '_model' in filename_stem else filename_stem
base_viz_dir = "visualisations"
model_viz_subdir = f"{model_type_name}_visualisations"
main_output_dir = os.path.join(base_viz_dir, model_folder_name, model_viz_subdir)

os.makedirs(main_output_dir, exist_ok=True)

print(f"Visualizations will be saved to: {main_output_dir}")

try:
    print(f"Loading model from: {model_load_path} to device: {device}")
    model.load_state_dict(torch.load(model_load_path, map_location=device))
    model.to(device) 

    model.eval()
    with torch.no_grad():
        for idx, (inputs, targets) in enumerate(test_loader):
            inputs = inputs.to(device)
            targets = targets.to(device)

            inputs_for_model = inputs.permute(0, 2, 1, 3, 4) 
            outputs = model(inputs_for_model)

            input_imgs  = tensor_to_numpy(inputs, is_input_sequence=True)
            target_img = tensor_to_numpy(targets, is_input_sequence=False)
            first_predicted_frame = outputs[:, :, 0:1, :, :] 

            output_img = tensor_to_numpy(first_predicted_frame, is_input_sequence=False)


            # Compute the difference (target - prediction) for each channel.
            # This now works as both target_img and output_img are (H, W, 2)
            diff_img = target_img - output_img

            # Create a sub-directory for the current sample inside the main output directory
            sample_output_dir = os.path.join(main_output_dir, f"sample_{idx}")
            os.makedirs(sample_output_dir, exist_ok=True)

            # Determine the number of input frames
            S_in = input_imgs.shape[0]
            # Total rows needed: Input frames + Target + Pred + Diff
            total_rows_needed = S_in + 3

            fig, axs = plt.subplots(total_rows_needed, 2, figsize=(10, 4 * total_rows_needed)) 
            for i in range(S_in):
                axs[i, 0].imshow(input_imgs[i, ..., 0], cmap='gray')
                axs[i, 0].set_title(f"Sample {idx}: Input Frame {i} Flow X")
                axs[i, 1].imshow(input_imgs[i, ..., 1], cmap='gray')
                axs[i, 1].set_title(f"Sample {idx}: Input Frame {i} Flow Y")

            target_row = S_in
            axs[target_row, 0].imshow(target_img[..., 0], cmap='gray')
            axs[target_row, 0].set_title(f"Sample {idx}: Target Flow X")
            axs[target_row, 1].imshow(target_img[..., 1], cmap='gray')
            axs[target_row, 1].set_title(f"Sample {idx}: Target Flow Y")

            pred_row = target_row + 1
            axs[pred_row, 0].imshow(output_img[..., 0], cmap='gray')
            axs[pred_row, 0].set_title(f"Sample {idx}: Prediction (Frame 0) Flow X") 
            axs[pred_row, 1].imshow(output_img[..., 1], cmap='gray')
            axs[pred_row, 1].set_title(f"Sample {idx}: Prediction (Frame 0) Flow Y") 

            diff_row = pred_row + 1
            im0 = axs[diff_row, 0].imshow(diff_img[..., 0])
            axs[diff_row, 0].set_title(f"Sample {idx}: Diff (Target - Pred Frame 0) Flow X") 
            fig.colorbar(im0, ax=axs[diff_row, 0])

            im1 = axs[diff_row, 1].imshow(diff_img[..., 1],)
            axs[diff_row, 1].set_title(f"Sample {idx}: Diff (Target - Pred Frame 0) Flow Y") 
            fig.colorbar(im1, ax=axs[diff_row, 1])

            for ax in axs.flat:
                ax.set_xticks([])
                ax.set_yticks([])

            plt.tight_layout()
            fig.canvas.draw() 

            axes_info = []
            for i in range(S_in): # Input frames
                axes_info.append((axs[i, 0], f'input_frame_{i}_flow_x.png'))
                axes_info.append((axs[i, 1], f'input_frame_{i}_flow_y.png'))

            # Target, Prediction (Frame 0), Difference - Use calculated row indices
            axes_info.extend([
                (axs[target_row, 0], 'target_flow_x.png'),
                (axs[target_row, 1], 'target_flow_y.png'),
                (axs[pred_row, 0], 'prediction_frame_0_flow_x.png'),
                (axs[pred_row, 1], 'prediction_frame_0_flow_y.png'), 
                (axs[diff_row, 0], 'diff_target_pred_frame_0_flow_x.png'),
                (axs[diff_row, 1], 'diff_target_pred_frame_0_flow_y.png'),
            ])


            # Get the bounding boxes of each axis in display (pixel) coordinates
            bbox_list = [ax.get_tightbbox(fig.canvas.get_renderer()) for ax, _ in axes_info]

            # Define the path for the combined image within the sample directory
            combined_save_path = os.path.join(sample_output_dir, "combined_visualization.png")

            try:
                 fig.savefig(combined_save_path, dpi=fig.dpi) 
            except Exception as save_err:
                 print(f"Error saving combined figure for sample {idx}: {save_err}")


            plt.close(fig)

            combined_img = None 
            try:
                if os.path.exists(combined_save_path): 
                    combined_img = Image.open(combined_save_path)
                    img_width, img_height = combined_img.size

                    for ((ax_info, filename), bbox) in zip(axes_info, bbox_list):
                        [[x0, y0], [x1, y1]] = bbox.get_points()

                        left = int(round(x0))
                        upper = int(round(img_height - y1)) 
                        right = int(round(x1))
                        lower = int(round(img_height - y0)) 

                        left = max(0, left)
                        upper = max(0, upper)
                        right = min(img_width, right)
                        lower = min(img_height, lower)

                        # Ensure bounding box is valid before cropping
                        if right > left and lower > upper:
                            cropped_img = combined_img.crop((left, upper, right, lower))
                            individual_save_path = os.path.join(sample_output_dir, filename)
                            cropped_img.save(individual_save_path)
                            cropped_img.close()
                        else:
                            print(f"Warning: Invalid bounding box [{left},{upper},{right},{lower}] for sample {idx}, file {filename} (img size: {img_width}x{img_height}). Skipping crop.")


            except Exception as crop_err:
                 print(f"Error processing or cropping combined image for sample {idx}: {crop_err}")
                 import traceback
                 traceback.print_exc()
            finally:
                if combined_img:
                    combined_img.close()


    print(f"Combined and individual visualizations saved to subdirectories within '{main_output_dir}'.")

except Exception as e:
    # Catch broader exceptions including OSError for environment issues or other errors
    print(f"An error occurred during processing: {e}")
    print("Please check the error message and ensure your environment is correctly set up (e.g., CUDA, dependencies).")
    import traceback
    traceback.print_exc() # Print detailed traceback

Visualizations will be saved to: visualisations/love_parade_3_length/fno_visualisations
Loading model from: /cs/student/projects1/2021/rstewart/code/models/love_parade_3_length/fno_model_3.pth to device: cuda


/tmp/ipykernel_2732452/829392219.py:102: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_load_path, map_location=device))


Combined and individual visualizations saved to subdirectories within 'visualisations/love_parade_3_length/fno_visualisations'.
